In [ ]:
import pickle
import matplotlib.pyplot as plt # creating visualizations
import numpy as np # basic math and random numbers
import torch # package for building functions with learnable parameters
import torch.nn as nn # prebuilt functions specific to neural networks
from torch.autograd import Variable # storing data while learning
#from disp import ADE,FDE,prediction_displacement,ADE_double_coordinates,FDE_double_coordinates,prediction_displacement_double

In [ ]:
class MDN_bivirate_old(nn.Module):
    def __init__(self,n_hidden, n_gaussians,input_shape = 16):
        super(MDN_bivirate_old, self).__init__()
        self.flatten = nn.Flatten(start_dim=1)
        self.z_h = nn.Sequential(
            nn.Linear(input_shape, n_hidden),
            nn.Tanh()
        )
        self.z_pi = nn.Linear(n_hidden, n_gaussians).to(device)
        self.z_sigma_x = nn.Linear(n_hidden, n_gaussians).to(device)
        self.z_sigma_y = nn.Linear(n_hidden, n_gaussians).to(device)
        self.z_mu_x = nn.Linear(n_hidden, n_gaussians).to(device)  
        self.z_mu_y = nn.Linear(n_hidden, n_gaussians).to(device)
    
    def diagonal(self,x):
        """Average first and last element of a 1-D array"""
        return np.diag(np.full(2,[x[0],x[1]]))

    def forward(self, x):
        # print("x.shape: ",x.shape)
        x = self.flatten(x)
        z_h = self.z_h(x).to(device)
        pi = nn.functional.softmax(self.z_pi(z_h), -1).to(device)
        sigma_x = torch.exp(self.z_sigma_x(z_h)).to(device)
        sigma_y = torch.exp(self.z_sigma_y(z_h)).to(device)
        mu_x = self.z_mu_x(z_h).to(device)
        mu_y = self.z_mu_y(z_h).to(device)
        
        # reshape to concatinate
        mux_detached = mu_x.detach().cpu().numpy().reshape(mu_x.shape[0],mu_x.shape[1],1)
        muy_detached = mu_y.detach().cpu().numpy().reshape(mu_y.shape[0],mu_x.shape[1],1)
        sigmax_detached = sigma_x.detach().cpu().numpy().reshape((sigma_x.shape[0],sigma_x.shape[1],1))
        sigmay_detached = sigma_x.detach().cpu().numpy().reshape((sigma_y.shape[0],sigma_y.shape[1],1))
        # concatinate all mu and sigma values
        # mu = torch.tensor(np.concatenate((mux_detached, muy_detached), axis=2)).to(device)
        # sigma_xy = np.concatenate((sigmax_detached,sigmay_detached), axis=2)
        mu =np.concatenate((mux_detached, muy_detached), axis=2)
        sigma_xy = np.concatenate((sigmax_detached,sigmay_detached), axis=2)

        # Change sigma to diagonal covariance matrix
        axis = 2
        #sigma = torch.tensor(np.apply_along_axis(self.diagonal,axis,sigma_xy)).to(device)
 
        sigma = np.apply_along_axis(self.diagonal,axis,sigma_xy)
        # print("sigma.shape: ",sigma.shape)
        return pi, sigma, mu

In [ ]:
def mdn_loss_fn_old(pis, mus, sigmas, y):
    results = []
    # calculate the score for each mixture of the gaussian_distribution
    # input shape (sample_size,num_mixtures,parameter) parametr is 2 in mue (x,y) and 2,2 in sigma [xx,xy,yx,yy] 
    # Pi has shape of  (sample_size,num_mixtures)
    # swap axis to have shape (num_mixtures,sample_size,parameter)
    mus = np.swapaxes(mus,0,1)
    sigmas = np.swapaxes(sigmas,0,1)
    pis = pis.detach().cpu().numpy()
    pis = np.swapaxes(pis,0,1)
    for mu,sigma,pi in zip(mus,sigmas,pis):
        # print(f"From for loop loss sigma: {sigma.shape}")
        result = multivariate_gaussian(y, mu, sigma) * pi
        results.append(result)
    result  = torch.tensor(results)
    result = np.swapaxes(result,0,1)
    # print("result1: ",result.shape)
    result = torch.sum(result, dim=1)
    # print("result after sum: ",result.shape)
    result = -torch.log(result)
    # print("result after log: ",result.shape)
    return torch.mean(result)

In [ ]:
def multivariate_gaussian(pos, mu, Sigma):
    """Return the multivariate Gaussian distribution on array pos.
    pos is an array constructed by packing the meshed arrays of variables
    x_1, x_2, x_3, ..., x_k into its _last_ dimension.
    """
   # Bivariate
    n = 2
    dims = Sigma.ndim
    if dims > 2:
        results = []
        for pos_i,sigma_i,mu_i in zip(pos,Sigma,mu):
            Sigma_det = np.linalg.det(sigma_i)
            Sigma_inv = np.linalg.inv(sigma_i)
            N = np.sqrt((2*np.pi)**n * Sigma_det)
            pos_mu = pos_i- mu_i
           #print(f"mu.shape: {mu_i.shape}, Sigma: {sigma_i.shape} , N.shape: {N},pos.shape {pos_i.shape} pos_mu.shape {pos_mu.shape}")
            fac = np.einsum('...k,kl,...l->...', pos_mu, Sigma_inv, pos_mu)
            result_i = np.exp(-fac / 2) / N
            results.append(result_i)
        
        return np.array(results) 

    else:
        Sigma_det = np.linalg.det(Sigma)
        Sigma_inv = np.linalg.inv(Sigma)
        N = np.sqrt((2*np.pi)**n * Sigma_det)
        # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
        # way across all the input variables.
        # print("Multi-Dimentional",pos.device)
        pos_mu = pos.detach().cpu().numpy()-mu
        #print(f"mu.shape: {mu.shape}, Sigma: {Sigma.shape} , N.Sigma_det: {Sigma_det.shape},pos.shape {pos.shape} pos_mu.shape {pos_mu.shape}")
        fac = np.einsum('...k,kl,...l->...', pos_mu, Sigma_inv, pos_mu)

        return np.exp(-fac / 2) / N
        
